<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [21]</a>'.</span>

## Comparisons

This code can execute the following variants of losses:  

1. **Variant 1:**  Purely Physics  
   $L_{\theta} = L_{\text{PDE}}$  
   Use $n_{\text{used}} = 0$  

2. **Variant 2:**  Physics + Data  
   $L_{\theta} = L_{\text{PDE}} + \Sigma_{i=1}^{n_{\text{used}}}\| u_i - \hat{u}_i \|_2^2$  
   Use $n_{\text{used}} \in (0, 500]$  

All the results presented were obtained as follows:
1. By estimating the gradients in the physics-informed loss terms using forward mode automatic differentiation (AD).
2. The output field values at given grid points were computed in one forward pass of the network using the einsum function.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torchsummary import summary
import torch.distributions as td
import math
from sklearn import metrics
import argparse
import random
import os
import time 
import matplotlib.pyplot as plt
from termcolor import colored


import sys
sys.path.append("../..")

from utils.networks import *
from utils.visualizer_misc import *
from utils.forward_autodiff import *
from utils.misc import *

from utils.deeponet_networks_2d import *
from utils.misc_stove import *
from utils.visualizer_stove import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Tag this cell with 'parameters'
# parameters
seed = 0 # Seed number.
n_used = 0 # Ensure n_used is a multiple of 10 (as group size is 10) # Number of full training fields used for estimating the data-driven loss term
n_iterations = 1000 # Number of iterations.
save = False # Save results.

In [3]:
if save == True:
    resultdir = os.path.join(os.getcwd(),'results','ForwardAD_neval_c','a_Vanilla-NO_ForwardADeinsum') 
    if not os.path.exists(resultdir):
        os.makedirs(resultdir)
else:
    resultdir = None

In [4]:
set_seed(seed)

seed = 0


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
metadata_np = np.load(os.path.join('..','..','data/2D_Stove/metadata.npz'), allow_pickle=True)
metadata = convert_metadata_to_torch(metadata_np, device)
for key, value in metadata.items():
        if isinstance(value, torch.Tensor):
            print(f"Shape of {key}: {value.shape}")
        elif isinstance(value, dict):
            print(f"{key} is a dictionary with {len(value)} keys.")

Shape of t_span: torch.Size([20])
Shape of x_span: torch.Size([64, 64])
Shape of y_span: torch.Size([64, 64])
shapes is a dictionary with 10 keys.


In [7]:
t_span, x_span, y_span = metadata['t_span'], metadata['x_span'], metadata['y_span']

nt, nx, ny = len(t_span), len(x_span), len(y_span) # number of discretizations in time, location_x and location_y.
print("nt =",nt, ", nx =",nx, "ny =",ny)
print("Shape of t-span, x-span, and y-span:",t_span.shape, x_span.shape, y_span.shape)
print("t-span:", t_span)
print("x-span:", x_span)
print("y-span:", y_span)

L = 2.         # Simulation domain [-L, L]^2
T = 1.         # Simulation time
D_value = 1.0  # Diffusion coefficient  

grid = torch.vstack((t_span.repeat_interleave(ny*nx), 
              x_span.flatten().repeat(nt),
              y_span.flatten().repeat(nt))).T
print("Shape of grid:", grid.shape) # (nt*nx*ny, 3)
print("grid:", grid) # time, location_x, location_y

nt = 20 , nx = 64 ny = 64
Shape of t-span, x-span, and y-span: torch.Size([20]) torch.Size([64, 64]) torch.Size([64, 64])
t-span: tensor([0.0500, 0.1000, 0.1500, 0.2000, 0.2500, 0.3000, 0.3500, 0.4000, 0.4500,
        0.5000, 0.5500, 0.6000, 0.6500, 0.7000, 0.7500, 0.8000, 0.8500, 0.9000,
        0.9500, 1.0000], device='cuda:0')
x-span: tensor([[-1.9688, -1.9062, -1.8438,  ...,  1.8438,  1.9062,  1.9688],
        [-1.9688, -1.9062, -1.8438,  ...,  1.8438,  1.9062,  1.9688],
        [-1.9688, -1.9062, -1.8438,  ...,  1.8438,  1.9062,  1.9688],
        ...,
        [-1.9688, -1.9062, -1.8438,  ...,  1.8438,  1.9062,  1.9688],
        [-1.9688, -1.9062, -1.8438,  ...,  1.8438,  1.9062,  1.9688],
        [-1.9688, -1.9062, -1.8438,  ...,  1.8438,  1.9062,  1.9688]],
       device='cuda:0')
y-span: tensor([[-1.9688, -1.9688, -1.9688,  ..., -1.9688, -1.9688, -1.9688],
        [-1.9062, -1.9062, -1.9062,  ..., -1.9062, -1.9062, -1.9062],
        [-1.8438, -1.8438, -1.8438,  ..., -1.8438, -1.

In [8]:
stove_full_solution_fields_groups_np = np.load(os.path.join('..','..','data/2D_Stove/stove_full_solution_fields.npz'), allow_pickle=True)
print(stove_full_solution_fields_groups_np.keys())

stove_source_fields_only_groups_np = np.load(os.path.join('..','..','data/2D_Stove/stove_source_fields_only.npz'), allow_pickle=True)
print(stove_source_fields_only_groups_np.keys())

KeysView(NpzFile '../../data/2D_Stove/stove_full_solution_fields.npz' with keys: full_solution_all_groups)
KeysView(NpzFile '../../data/2D_Stove/stove_source_fields_only.npz' with keys: only_sources_all_groups)


In [9]:
# Convert the NumPy groups to Pytorch
stove_full_solution_fields_groups = convert_groupdict_to_torch(stove_full_solution_fields_groups_np, device)
stove_source_fields_only_groups = convert_groupdict_to_torch(stove_source_fields_only_groups_np, device)

# Check the shapes
print('stove_full_solution_fields_groups:')
check_shapes(stove_full_solution_fields_groups)
print(colored('#' * 230, 'green'))

print('stove_source_fields_only_groups:')
check_shapes(stove_source_fields_only_groups)
print(colored('#' * 230, 'green'))

stove_full_solution_fields_groups:
Group 0:
  Input Parameters shape: torch.Size([10, 4])
  Input Samples shape: torch.Size([10, 64, 64])
  Output Samples shape: torch.Size([10, 20, 64, 64])
Group 1:
  Input Parameters shape: torch.Size([10, 4])
  Input Samples shape: torch.Size([10, 64, 64])
  Output Samples shape: torch.Size([10, 20, 64, 64])
######################################################################################################################################################################################################################################
stove_source_fields_only_groups:
Group 0:
  Input Parameters shape: torch.Size([10, 4])
  Input Samples shape: torch.Size([10, 64, 64])
Group 1:
  Input Parameters shape: torch.Size([10, 4])
  Input Samples shape: torch.Size([10, 64, 64])
#######################################################################################################################################################################################

In [10]:
# Split the data into training and testing groups
train_group, test_group = split_groups(stove_full_solution_fields_groups, seed, train_size=50, test_size=10)

# Print the shapes for verification
print("Training Group:")
print(colored('#' * 20, 'red'))
print_group_shapes(train_group)
print(colored('#' * 230, 'green'))
print("Testing Group:")
print(colored('#' * 20, 'red'))
print_group_shapes(test_group)
print(colored('#' * 230, 'green'))

Training Group:
####################
Group Data Shapes:
Group 4:
  Input Parameters shape: torch.Size([10, 4])
  Input Samples shape: torch.Size([10, 64, 64])
  Output Samples shape: torch.Size([10, 20, 64, 64])
--------------------------------------------------
Group 10:
  Input Parameters shape: torch.Size([10, 4])
  Input Samples shape: torch.Size([10, 64, 64])
  Output Samples shape: torch.Size([10, 20, 64, 64])
--------------------------------------------------
######################################################################################################################################################################################################################################
Testing Group:
####################
Group Data Shapes:
Group 26:
  Input Parameters shape: torch.Size([10, 4])
  Input Samples shape: torch.Size([10, 64, 64])
  Output Samples shape: torch.Size([10, 20, 64, 64])
--------------------------------------------------
Group 35:
  Input Parameters shape:

In [11]:
train_data = load_and_combine_groups(train_group, 'Train', combine=True, device=device)
input_parameters_train = train_data['input_parameters']
inputs_train = train_data['input_samples']
outputs_train = train_data['output_samples']

test_data = load_and_combine_groups(test_group, 'Test', combine=True, device=device)
input_parameters_test = test_data['input_parameters']
inputs_test = test_data['input_samples']
outputs_test = test_data['output_samples']

print(colored('#' * 20, 'red'))

# Check the shapes of the subsets
print("Shape of input_parameters_train:", input_parameters_train.shape)
print("Shape of input_parameters_test:", input_parameters_test.shape)
print("Shape of inputs_train:", inputs_train.shape)
print("Shape of inputs_test:", inputs_test.shape)
print("Shape of outputs_train:", outputs_train.shape)
print("Shape of outputs_test:", outputs_test.shape)
print(colored('#' * 20, 'green'))

Train Data Shapes:
  Input Parameters Shape: torch.Size([500, 4])
  Input Samples Shape: torch.Size([500, 64, 64])
  Output Samples Shape: torch.Size([500, 20, 64, 64])
Test Data Shapes:
  Input Parameters Shape: torch.Size([100, 4])
  Input Samples Shape: torch.Size([100, 64, 64])
  Output Samples Shape: torch.Size([100, 20, 64, 64])
####################
Shape of input_parameters_train: torch.Size([500, 4])
Shape of input_parameters_test: torch.Size([100, 4])
Shape of inputs_train: torch.Size([500, 64, 64])
Shape of inputs_test: torch.Size([100, 64, 64])
Shape of outputs_train: torch.Size([500, 20, 64, 64])
Shape of outputs_test: torch.Size([100, 20, 64, 64])
####################


In [12]:
stove_source_fields_only = load_and_combine_groups(stove_source_fields_only_groups, 'Source only', combine=True, device=device)

Source only Data Shapes:
  Input Parameters Shape: torch.Size([2000, 4])
  Input Samples Shape: torch.Size([2000, 64, 64])


In [13]:
# Of these full training fields available I am using only n_used fields for estimating the data-driven loss term 
input_parameters_train_used = input_parameters_train[:n_used, :]
print("Shape of input_parameters_train_used:", input_parameters_train_used.shape)
inputs_train_used = inputs_train[:n_used, :, :]
print("Shape of inputs_train_used:", inputs_train_used.shape)
outputs_train_used = outputs_train[:n_used, :, :, :]
print("Shape of outputs_train_used:", outputs_train_used.shape)

Shape of input_parameters_train_used: torch.Size([0, 4])
Shape of inputs_train_used: torch.Size([0, 64, 64])
Shape of outputs_train_used: torch.Size([0, 20, 64, 64])


In [14]:
"""
input_neurons_branch: Number of input neurons in the branch net.
input_neurons_trunk: Number of input neurons in the trunk net.
p: Number of output neurons in both the branch and trunk net.
"""

p = 128 # Number of output neurons in both the branch and trunk net.

input_neurons_branch = (ny, nx) # Specify input size of image as a tuple (height, width)
n_channels = 1
num_filters = [40, 60, 80, 100]
filter_sizes = [3, 3, 3, 3]
strides = [1]*len(num_filters)
paddings = [0]*len(num_filters)
poolings = [('avg', 2, 2), ('avg', 2, 2), ('avg', 2, 2), ('avg', 2, 2)]  # Pooling layer specification (type, kernel_size, stride)
end_MLP_layersizes = [150, 150, p]
activation = nn.ReLU() # nn.SiLU() #Sin() #nn.LeakyReLU() #nn.Tanh()
branch_net = ConvNet(input_neurons_branch, n_channels, num_filters, filter_sizes, strides, paddings, poolings, end_MLP_layersizes, activation)
branch_net.to(device)
# print(branch_net)
print('BRANCH-NET SUMMARY:')
summary(branch_net, input_size=(n_channels, ny, nx))  # input shape is (channels, height, width)
print('#'*100)

input_neurons_trunk = 3 # 3 corresponds to t, x and y
trunk_net = DenseNet(layersizes=[input_neurons_trunk] + [128]*4 + [p], activation=nn.SiLU()) #Sin() #nn.LeakyReLU() #nn.Tanh()
trunk_net.to(device)
# print(trunk_net)
print('TRUNK-NET SUMMARY:')
summary(trunk_net, input_size=(input_neurons_trunk,))
print('#'*100)

model = Vanilla_NO_model(branch_net, trunk_net)
model.to(device)

BRANCH-NET SUMMARY:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 40, 62, 62]             400
              ReLU-2           [-1, 40, 62, 62]               0
         AvgPool2d-3           [-1, 40, 31, 31]               0
            Conv2d-4           [-1, 60, 29, 29]          21,660
              ReLU-5           [-1, 60, 29, 29]               0
         AvgPool2d-6           [-1, 60, 14, 14]               0
            Conv2d-7           [-1, 80, 12, 12]          43,280
              ReLU-8           [-1, 80, 12, 12]               0
         AvgPool2d-9             [-1, 80, 6, 6]               0
           Conv2d-10            [-1, 100, 4, 4]          72,100
             ReLU-11            [-1, 100, 4, 4]               0
        AvgPool2d-12            [-1, 100, 2, 2]               0
          Flatten-13                  [-1, 400]               0
           Linear-1

Vanilla_NO_model(
  (branch_net): ConvNet(
    (model): Sequential(
      (0): Conv2d(1, 40, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (3): Conv2d(40, 60, kernel_size=(3, 3), stride=(1, 1))
      (4): ReLU()
      (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (6): Conv2d(60, 80, kernel_size=(3, 3), stride=(1, 1))
      (7): ReLU()
      (8): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (9): Conv2d(80, 100, kernel_size=(3, 3), stride=(1, 1))
      (10): ReLU()
      (11): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (12): Flatten(start_dim=1, end_dim=-1)
      (13): Linear(in_features=400, out_features=150, bias=True)
      (14): ReLU()
      (15): Linear(in_features=150, out_features=150, bias=True)
      (16): ReLU()
      (17): Linear(in_features=150, out_features=128, bias=True)
    )
  )
  (trunk_net): DenseNet(
    (activation): SiLU()
    (layers): ModuleList(
      (0): Linear(in_f

In [15]:
num_learnable_parameters = count_learnable_parameters(branch_net) + count_learnable_parameters(trunk_net)
print("Total number of learnable parameters:", num_learnable_parameters)

Total number of learnable parameters: 306128


In [16]:
def u_pred(net, inputs, t, x, y):
    factor = t*(x-(-L))*(x-L)*(y-(-L))*(y-L)/(T*(2*L)*(2*L)*(2*L)*(2*L)) # (neval, 1) # enforcing ICs and BCs in hard way
    factor_repeated = factor.T.repeat(inputs.shape[0], 1)# (bs, neval) # Repeat factor to match the batch size
    u = net(inputs, torch.hstack([t, x, y]))*factor_repeated # (bs, neval)
    return u

In [17]:
def loss_pde_residual(net, source_fields_parameters, source_fields, t, x, y):
    
    # Using forward automatic differention to estimate derivatives in the physics informed loss
    tangent_t, tangent_x, tangent_y = torch.ones(t.shape).to(device), torch.ones(x.shape).to(device), torch.ones(y.shape).to(device)
    ut  = FWDAD_first_order_derivative(lambda t: u_pred(net, source_fields, t, x, y), t, tangent_t)  # (bs, neval_c) 
    uxx = FWDAD_second_order_derivative(lambda x: u_pred(net, source_fields, t, x, y), x, tangent_x) # (bs, neval_c)
    uyy = FWDAD_second_order_derivative(lambda y: u_pred(net, source_fields, t, x, y), y, tangent_y) # (bs, neval_c)
    
    bs_ = source_fields.shape[0]
    sf_values_ = torch.zeros((bs_, x.shape[0], 1)).to(device)
    for j in range(bs_):
        source_class = Source(a=source_fields_parameters[j][3], r=source_fields_parameters[j][2], 
                      x=x, y=y, 
                      xc=0., yc=0.,
                      device=device)
        shape = get_key_from_value(shape_map, source_fields_parameters[j, 0])
        sf_values_[j] = source_class.type_source(shape, num_sides=int(source_fields_parameters[j, 1])) # source function: s(x, y) values
    sf_values = sf_values_.reshape(-1, x.shape[0]) # (bs, neval_c)
    
    pde_residual = (ut - (D_value*uxx) - (D_value*uyy) - sf_values)**2
    
    return torch.mean(pde_residual)

In [18]:
def collocation_points(tc_span, xc_span, yc_span, neval_dict):
    tc = tc_span.repeat_interleave(neval_dict['loc']).unsqueeze(-1)
    xc = xc_span.flatten().repeat(neval_dict['t']).unsqueeze(-1)
    yc = yc_span.flatten().repeat(neval_dict['t']).unsqueeze(-1)
    return tc, xc, yc

In [19]:
neval_t = 20  # Number of randomly chosen time points at which output field is evaluated.
neval_x = 64 
# neval_loc = neval_x*neval_y  # Number of locations at which output field is evaluated at each time point.

In [20]:
# Parameters
neval_t = 128
neval_x = 128


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [21]:
start_time = time.time()

bs = 128 # Batch size

neval_y = neval_x

neval_c = {'t': neval_t, 'loc': neval_x*neval_y}  # Number of collocation points within the domain.
        
# Training
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20000, gamma=0.1) # gamma=0.8

iteration_list, loss_list, learningrates_list = [], [], []
datadriven_loss_list, pinn_loss_list = [], []
test_iteration_list, test_loss_list = [], []

for iteration in range(n_iterations):
    
    if n_used == 0:
        datadriven_loss = torch.tensor([0.]).to(device)
        # print('*********')
    else:
        indices_datadriven = torch.randperm(n_used).to(device) # Generate random permutation of indices
        inputs_train_used_batch = inputs_train_used.reshape(-1, 1, ny, nx)[indices_datadriven[0:bs]]
        outputs_train_used_batch = outputs_train_used.reshape(-1, nt*nx*ny)[indices_datadriven[0:bs]]
        # print(f"Shape of inputs_train_used_batch:", inputs_train_used_batch.shape) # (bs, no. of channels, height, width)
        # print(f"Shape of outputs_train_used_batch:", outputs_train_used_batch.shape) # (bs, nt*nx*ny)

        predicted_values = u_pred(model, inputs_train_used_batch, 
                              grid[:, 0].reshape(-1,1), 
                              grid[:, 1].reshape(-1,1), 
                              grid[:, 2].reshape(-1,1))  # (bs, neval) = (bs, nt*nx*ny)
        target_values = outputs_train_used_batch # (bs, nt*nx*ny)
        datadriven_loss = nn.MSELoss()(predicted_values, target_values)
        # print('*********')
    
    num_samples = stove_source_fields_only['input_samples'].shape[0]
    indices_pinn = torch.randperm(num_samples).to(device) # Generate random permutation of indices
    input_parameters_batch = stove_source_fields_only['input_parameters'][indices_pinn[0:bs]]
    inputs_batch = stove_source_fields_only['input_samples'].reshape(-1, 1, ny, nx)[indices_pinn[0:bs]]
    # print(f"Shape of inputs_batch:", inputs_batch.shape) # (bs, no. of channels, height, width)

    # points within the domain
    tc_span = td.uniform.Uniform(0., T).sample((neval_c['t'], 1)).to(device)
    xc_span = td.uniform.Uniform(-L, L).sample((neval_c['loc'], 1)).to(device)
    yc_span = td.uniform.Uniform(-L, L).sample((neval_c['loc'], 1)).to(device)

    tc, xc, yc = collocation_points(tc_span, xc_span, yc_span, neval_c)
    pinn_loss = loss_pde_residual(model, input_parameters_batch, inputs_batch, tc, xc, yc)
    # print('*********')

    optimizer.zero_grad()
    loss = datadriven_loss + pinn_loss
    loss.backward()
    # torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=1.0)
    optimizer.step()
    scheduler.step()

    if iteration % 500 == 0:
        # Test loss calculation
        model.eval()  # Set model to evaluation mode
        with torch.no_grad():
            test_predicted_values = u_pred(model, inputs_test.reshape(-1, 1, ny, nx), 
                          grid[:, 0].reshape(-1,1), 
                          grid[:, 1].reshape(-1,1), 
                          grid[:, 2].reshape(-1,1))  # (bs, neval) = (bs, nt*nx*ny)
            test_loss = nn.MSELoss()(test_predicted_values, outputs_test.reshape(-1, nt*nx*ny))
            test_iteration_list.append(iteration)
            test_loss_list.append(test_loss.item())  
        model.train()  # Switch back to training mode
        print('Iteration %s -' % iteration, 'loss = %f,' % loss,
              'data-driven loss = %f,' % datadriven_loss,'pinn loss = %f,' % pinn_loss,
              'learning rate = %f,' % optimizer.state_dict()['param_groups'][0]['lr'],
              'test loss = %f' % test_loss)

    iteration_list.append(iteration)
    loss_list.append(loss.item())
    datadriven_loss_list.append(datadriven_loss.item())
    pinn_loss_list.append(pinn_loss.item())
    learningrates_list.append(optimizer.state_dict()['param_groups'][0]['lr'])

if device.type == "cuda":
    free, total = torch.cuda.mem_get_info()
    mem_used_MB = (total - free) / 1024 ** 2
    print(f"Memory used: {mem_used_MB:.2f} MB")
    
if save == True:
    np.save(os.path.join(resultdir,'iteration_list.npy'), np.asarray(iteration_list))
    np.save(os.path.join(resultdir,'loss_list.npy'), np.asarray(loss_list))
    np.save(os.path.join(resultdir, 'datadriven_loss_list.npy'), np.asarray(datadriven_loss_list))
    np.save(os.path.join(resultdir, 'pinn_loss_list.npy'), np.asarray(pinn_loss_list))
    np.save(os.path.join(resultdir,'learningrates_list.npy'), np.asarray(learningrates_list))
    np.save(os.path.join(resultdir,'test_iteration_list.npy'), np.asarray(test_iteration_list))
    np.save(os.path.join(resultdir, 'test_loss_list.npy'), np.asarray(test_loss_list)) 

plot_loss_terms(resultdir, iteration_list, loss_list, datadriven_loss_list, pinn_loss_list, save)  
    
plot_training_loss(resultdir, iteration_list, loss_list, save) 

plot_testing_loss(resultdir, test_iteration_list, test_loss_list, save)

plot_training_testing_loss(resultdir, iteration_list, loss_list, test_iteration_list, test_loss_list, save)

plot_learningrates(resultdir, iteration_list, learningrates_list, save)  
    
# end timer
end_time = time.time()
training_time = end_time - start_time

runtime_per_iter = training_time/n_iterations # in sec/iter

OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB. GPU 0 has a total capacity of 79.14 GiB of which 820.75 MiB is free. Including non-PyTorch memory, this process has 78.33 GiB memory in use. Of the allocated memory 76.92 GiB is allocated by PyTorch, and 934.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
if save == True:
    torch.save(model.state_dict(), os.path.join(resultdir,'model_state_dict.pt'))
# model.load_state_dict(torch.load(os.path.join(resultdir, 'model_state_dict.pt'), map_location=device))

In [ ]:
# Predictions
predictions_test = u_pred(model, inputs_test.reshape(-1, 1, ny, nx), 
                          grid[:, 0].reshape(-1,1), 
                          grid[:, 1].reshape(-1,1), 
                          grid[:, 2].reshape(-1,1))  # (bs, neval) = (bs, nt*nx*ny)
# print(predictions_test.shape)

mse_list, r2score_list, relerror_list = [], [], []
    
for i in range(inputs_test.shape[0]):
    
    prediction_i = predictions_test[i].reshape(1, -1) # (1, nt*nx*ny)
    target_i = outputs_test[i].reshape(1, -1) # (1, nt*nx*ny)

    mse_i = F.mse_loss(prediction_i.cpu(), target_i.cpu())
    r2score_i = metrics.r2_score(target_i.flatten().cpu().detach().numpy(), prediction_i.flatten().cpu().detach().numpy()) 
    relerror_i = np.linalg.norm(target_i.flatten().cpu().detach().numpy() - prediction_i.flatten().cpu().detach().numpy()) / np.linalg.norm(target_i.flatten().cpu().detach().numpy())
        
    mse_list.append(mse_i.item())
    r2score_list.append(r2score_i.item())
    relerror_list.append(relerror_i.item())
    
    # Plot the full solution-field for few cases (2 groups i.e., 10*2=20):
    if (i+1) <= 20:
        print(colored('TEST SAMPLE '+str(i+1), 'red'))
        shape = get_key_from_value(shape_map, input_parameters_test[i, 0])
        print(colored(f"Shape = {shape}, r = {input_parameters_test[i,2]:.3f}, a_value = {input_parameters_test[i,3]:.3f}", 'red'))
        
        r2score_i = float('%.4f'%r2score_i)
        relerror_i = float('%.4f'%relerror_i)
        print('Rel. L2 Error = '+str(relerror_i)+', R2 score = '+str(r2score_i))
        
        # Plotting 
        plot_source(i, x_span, y_span, inputs_test[i], f"{shape.capitalize()} Source", 'hot', resultdir, save)
        
        cmap = 'hot'  # Color map
        fontsize = 14  # Font size for labels and titles
        levels = 100
        plot_solution(i, x_span, y_span, target_i.reshape(nt,ny,nx), t_span, f"True Solution for {shape.capitalize()} Source", cmap, fontsize, levels, resultdir, save, 'True-Solution')
        plot_solution(i, x_span, y_span, prediction_i.reshape(nt,ny,nx), t_span, f"Predicted Solution for {shape.capitalize()} Source", cmap, fontsize, levels, resultdir, save, 'Predicted-Solution')
        plot_solution(i, x_span, y_span, torch.abs(target_i.reshape(nt,ny,nx) - prediction_i.reshape(nt,ny,nx)), t_span, f"Absolute error for {shape.capitalize()} Source", cmap, fontsize, levels, resultdir, save, 'Absolute error')
        print(colored('#'*230, 'green'))

mse = sum(mse_list) / len(mse_list)
print("Mean Squared Error Test:\n", mse)
r2score = sum(r2score_list) / len(r2score_list)
print("R2 score Test:\n", r2score)
relerror = sum(relerror_list) / len(relerror_list)
print("Rel. L2 Error Test:\n", relerror)

In [ ]:
test_dict = {
    "input_parameters_test": input_parameters_test.cpu(),
    "inputs_test": inputs_test.cpu(),
    "outputs_test": outputs_test.cpu(),
    "predictions_test": predictions_test.reshape(-1, nt, ny, nx).cpu()
}
for key, value in test_dict.items():
    print(f"Shape of {key}: {value.shape}")
print(colored('#'*230, 'green'))

if save == True:
    torch.save(test_dict, os.path.join(resultdir,'test_dict.pth'))

In [ ]:
performance_metrics(mse, r2score, relerror, training_time, runtime_per_iter, resultdir, save)
# GPU memory used
if device.type == "cuda":
    print(f"Memory used (in MB):\n{mem_used_MB:.2f}")